In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import gdown
import os
import zipfile
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
# Google Drive file URL
file_url = "https://drive.google.com/uc?id=1vHCvd9ZFkY9lfNwUnnMUbDQZjjpby8lv"

# Specify the output file path (the path where you want to save the file)
output_file = "disease.zip"  # Replace with actual file name

# Download the file
gdown.download(file_url, output_file, quiet=False)

zip_file = "disease.zip"  # Name of the downloaded file

# Make sure output directory exists
output_dir = "/content/extracted_file"
os.makedirs(output_dir, exist_ok=True)

# Extract the contents
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(output_dir)

Downloading...
From (original): https://drive.google.com/uc?id=1QwItNnETzlWHgZ89EbHFYg04ijkDszF6
From (redirected): https://drive.google.com/uc?id=1QwItNnETzlWHgZ89EbHFYg04ijkDszF6&confirm=t&uuid=effce612-230e-4675-b88d-127904889a07
To: /content/disease.zip
100%|██████████| 3.85G/3.85G [01:06<00:00, 58.2MB/s]


In [ ]:
# Path to main train folder that has many class folders
data_dir = "/content/extracted_file/Retinal Fundus Images - Copy/train"

# Specify the exact classes you want to include
target_classes = ['Pathological_Myopia', 'Normal_Fundus']

# Image settings
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

# Create generators with only selected classes
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.1)

train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    classes=target_classes,
    class_mode='sparse',
    subset='training',
    shuffle=True
)

val_generator = datagen.flow_from_directory(
    data_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    classes=target_classes,
    class_mode='sparse',
    subset='validation',
    shuffle=False
)

# Load VGG16 without top layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add classification head
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(128, activation='relu')(x)
output = Dense(2, activation='softmax')(x)  # 2 classes (softmax)

model = Model(inputs=base_model.input, outputs=output)

# Fine-tune last few layers of VGG16
for layer in base_model.layers[:-10]:
    layer.trainable = False
for layer in base_model.layers[-10:]:
    layer.trainable = True

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Train the model
model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=5,
)

Found 17024 images belonging to 2 classes.
Found 1891 images belonging to 2 classes.
Epoch 1/5
532/532 ━━━━━━━━━━━━━━━━━━━━ 198s 319ms/step - accuracy: 0.9552 - loss: 0.1007 - val_accuracy: 0.9820 - val_loss: 0.0382
Epoch 2/5
532/532 ━━━━━━━━━━━━━━━━━━━━ 162s 305ms/step - accuracy: 0.9895 - loss: 0.0297 - val_accuracy: 0.9894 - val_loss: 0.0246
Epoch 3/5
532/532 ━━━━━━━━━━━━━━━━━━━━ 203s 306ms/step - accuracy: 0.9969 - loss: 0.0087 - val_accuracy: 0.9984 - val_loss: 0.0146
Epoch 4/5
532/532 ━━━━━━━━━━━━━━━━━━━━ 162s 304ms/step - accuracy: 0.9954 - loss: 0.0103 - val_accuracy: 0.9995 - val_loss: 0.0079
Epoch 5/5
532/532 ━━━━━━━━━━━━━━━━━━━━ 162s 304ms/step - accuracy: 0.9996 - loss: 0.0011 - val_accuracy: 0.9995 - val_loss: 0.0022


In [ ]:
model.save("Pathological_Myopia.keras")